In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 10000)

import seaborn as sns

from sklearn.cluster import KMeans
import plotly.express as px

import xgboost as xgb

from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [ ]:
sales = pd.read_excel("sales_data.xlsx")

In [ ]:
sales

# EXPLATORY DATA ANALYSIS

# we will visualize sales for 1 item and 1 store

In [ ]:
sales['Date'].min()

In [ ]:
sales['Date'].max()

We will create a table containing date and other variables related to date like "day of week" and "week" to visualize

In [ ]:
item='Item 1'
store='Store A'

In [ ]:
sales2=sales[sales['Item']==item]
sales2=sales2[sales2['Store']==store]
sales2['Date'] =  pd.to_datetime(sales2['Date'], dayfirst=True)
def create_date_table(start, end):
        df = pd.DataFrame({"Date": pd.date_range(start, end)})
        df["Amount"] = 0
        df["Day"] = df.Date.dt.day_name()
        df["DayOfWeek"] = df.Date.dt.weekday
        df["Week"] = df.Date.dt.weekofyear
        df["Quarter"] = df.Date.dt.quarter
        df["Year"] = df.Date.dt.year
        df["Month"] = df.Date.dt.month
        df["DayOfMonth"] = df.Date.dt.day
        df["Year_half"] = (df.Quarter + 1) // 2
        return df
    
datedata=create_date_table('03.01.2020','30.08.2020')    
dailysales=datedata.merge(sales2, on="Date", how='left')
dailysales2=dailysales[['Date','DayOfWeek','Week','Year','Month','DayOfMonth','Quantity']]
dailysales3=dailysales2.groupby(['Date','DayOfWeek','Week','Year','Month','DayOfMonth'])['Quantity'].sum().reset_index(name ='Quantity')
dailysales3.index = dailysales3['Date']    
    
start_date = pd.Timestamp('2020-05-01')
end_date = pd.Timestamp('2020-06-01')
dailysales3['Covid19'] = dailysales3['Date'].apply(lambda x: 1 if  (x > start_date) & (x <= end_date ) else 0 )
    
del dailysales3['Date']

dailysales3['Quantity']=dailysales3['Quantity'].fillna(0)
dailysales3['Weekend'] = dailysales3['DayOfWeek'].apply(lambda x: 1 if  (x == 5) | (x == 6 ) else 0 )

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(data=dailysales3,x=dailysales3.index,y=dailysales3['Quantity'],color="r").set(title='Sales Over Date')

In [ ]:
dailyreceipt=dailysales3.groupby(['DayOfWeek'])['Quantity'].mean().reset_index(name ='Quantity')
plt.figure(figsize=(16,6))
sns.barplot(data=dailyreceipt,x=dailyreceipt['DayOfWeek'],y=dailyreceipt['Quantity']).set(title='Day Of Week Sales Distrubition')

We see that on thursday and friday sales increases

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(data=dailysales3,x=dailysales3.DayOfMonth,y=dailysales3['Quantity'],color="r").set(title='Day Of Month Sales Distrubition')

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(data=dailysales3,x=dailysales3.DayOfWeek,y=dailysales3['Quantity'],hue=dailysales3.Weekend).set(title='Compare Weekday and Weekend Sales')

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(data=dailysales3,x=dailysales3["DayOfMonth"],y=dailysales3['Quantity']).set(title='Boxplot of sales over  Day Of Month')

We see that the last week of month sales increase

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(data=dailysales3,x=dailysales3["DayOfWeek"],y=dailysales3['Quantity'],hue=dailysales3["Covid19"]).set(title='Visualize effect of Covid19 to sales')

In [ ]:
plt.figure(figsize=(16,6))
sns.lineplot(data=dailysales3,x=dailysales3['DayOfWeek'],y=dailysales3['Quantity'],hue=dailysales3.Covid19)

We see that through Covid19 pick period sales has decreased but it keep the day of week trend the same

# CLUSTER OF EACH ITEM ON EACH STORE

In [ ]:
#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [ ]:
sum_sales = sales.groupby(['Item','Store'])["Quantity"].sum().reset_index().sort_values(['Item',"Quantity"], ascending=[True, False])
#we will take into account items which has ben sold at least 8 stores 
count=sum_sales.groupby(['Item'])["Quantity"].count().reset_index().sort_values(by="Quantity",ascending=False)
count=count[count["Quantity"]>=8]
itemlist=count['Item'].unique()
columns = ['Item','Store',"Quantity"]
df_ = pd.DataFrame(columns=columns)
for i in itemlist:
    sum_sales_item=sum_sales[sum_sales['Item']==i]
    f_1 = sum_sales_item["Quantity"].values
    X=np.array(list(zip(f_1)))
    distortions = []
    K = range(1,10)
    for k in K:
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(X)
        distortions.append(kmeanModel.inertia_)
    plt.figure(figsize=(16,8))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title(i + ' The Elbow Method showing the optimal k')
    plt.show()
    kmeans=KMeans(n_clusters=4, init='k-means++').fit(X)
    labels=kmeans.predict(X)
    centroids=kmeans.cluster_centers_
    sum_sales_item['Cluster'] = kmeans.predict(X)
    #merge two dataframes
    sum_sales_item = order_cluster('Cluster',"Quantity",sum_sales_item,False)
    sum_sales_item['Cluster'].replace(0,'A',inplace=True)
    sum_sales_item['Cluster'].replace(1,'B',inplace=True)
    sum_sales_item['Cluster'].replace(2,'C',inplace=True)
    sum_sales_item['Cluster'].replace(3,'D',inplace=True)
    fig = px.bar(sum_sales_item, x='Store', y="Quantity",
              color='Cluster',title=i + " Sales Graph",
              height=500)
    fig.show()
    df_=df_.append(sum_sales_item)

In [ ]:
df_

# SALES PREDICTION WITH XGBOOST

In [ ]:
items =sales['Item'].unique()
stores=sales['Store'].unique()

In [ ]:
appended_data = []
for k in items:
    for i in stores:
        sales2=sales[sales['Item']==k]
        sales2=sales2[sales2['Store']==i]
        sales2['Date'] =  pd.to_datetime(sales2['Date'], dayfirst=True)
        def create_date_table(start, end):
            df = pd.DataFrame({"Date": pd.date_range(start, end)})
            df["Amount"] = 0
            df["Day"] = df.Date.dt.day_name()
            df["DayOfWeek"] = df.Date.dt.weekday
            df["Week"] = df.Date.dt.weekofyear
            df["Quarter"] = df.Date.dt.quarter
            df["Year"] = df.Date.dt.year
            df["Month"] = df.Date.dt.month
            df["DayOfMonth"] = df.Date.dt.day
            df["Year_half"] = (df.Quarter + 1) // 2
            return df
    
        datedata=create_date_table('03.01.2020','31.08.2020')    
        dailysales=datedata.merge(sales2, on="Date", how='left')
        dailysales2=dailysales[['Date','DayOfWeek','Week','Year','Month','DayOfMonth','Quantity']]
        dailysales3=dailysales2.groupby(['Date','DayOfWeek','Week','Year','Month','DayOfMonth'])['Quantity'].sum().reset_index(name ='Quantity')
        dailysales3.index = dailysales3['Date']    
    
        start_date = pd.Timestamp('2020-05-01')
        end_date = pd.Timestamp('2020-06-01')
        dailysales3['Covid19'] = dailysales3['Date'].apply(lambda x: 1 if  (x > start_date) & (x <= end_date ) else 0 )
    
        del dailysales3['Date']

        dailysales3['Quantity']=dailysales3['Quantity'].fillna(0)
        dailysales3['Weekend'] = dailysales3['DayOfWeek'].apply(lambda x: 1 if  (x == 5) | (x == 6 ) else 0 )

        plt.figure(figsize=(10,3))
        dailysales3.plot(subplots=True)
    
        data=dailysales3.copy()
        fig = px.line(data, x=data.index, y="Quantity", title=' Before outliers smoothing'+ i )
        fig.show()
    
        from scipy import stats
        data['z_score']=stats.zscore(data['Quantity'])
        fig = px.histogram(data, x="z_score",nbins=10)
        fig.show()
    
        data['Quantity_Es']=data['Quantity']
        data['Quantity_Es'][data.z_score > 3] = data['Quantity'].ewm(alpha=0.2).mean()
        data['Quantity_Es'][data.z_score < -2] = data['Quantity'].ewm(alpha=0.2).mean()
        fig = px.line(data, x=data.index, y="Quantity_Es", title=' After outliers smoothing'+ i )
        fig.show()
    
        data=data.drop(['Quantity', 'z_score'], axis=1)
        data_df=data.copy()
        data_df['item_month_mean'] = data_df.groupby(['Month'])['Quantity_Es'].transform('mean')       
        data_df['itemweekday_mean'] = data_df.groupby(['DayOfWeek'])['Quantity_Es'].transform('mean')
        data_df['Previous'] = data_df.groupby('DayOfWeek')['Quantity_Es'].shift()
        data_df['rolling_mean_7_days']=data_df['Quantity_Es'].rolling(7).apply(np.nanmean)
        data_df.rolling_mean_7_days=data_df.rolling_mean_7_days.shift(1)
        data_df['previous_day']=data_df['Quantity_Es'].shift(1)
        data_df_2=data_df.dropna()
    
        regressor_xgboost=xgb.XGBRegressor(objective='reg:squarederror',eval_metric ="mae")
        
        x=data_df_2[data_df_2.index <= "2020-08-30"]
        test = data_df_2[data_df_2.index > "2020-08-30"]
        test = test[test.index <= "2020-08-31"]

        y=x['Quantity_Es']
    
        x.drop(['Quantity_Es'],axis=1,inplace=True)
        test.drop(['Quantity_Es'],axis=1,inplace=True)
        
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1) # 70% training and 30% test
        
        #parameters = {'min_child_weight': [1,2,3],
               #'max_depth':  [2,3,4],
               #'colsample_bytree':  [0.1,0.3,0.5],
              #'n_estimators':  [500,800],
              #'alpha': [3,5,8]
                #}
        
        parameters = {'min_child_weight': [1,2],
               'max_depth':  [2,3],
               'colsample_bytree':  [0.3,0.5],
              'n_estimators':  [500,800],
              'alpha': [3,5]
                }
            
            
        grid_xgboost = GridSearchCV(estimator=regressor_xgboost,param_grid=parameters) 
        grid_xgboost.fit(X_train,y_train)
    
        print( 'best score: {0:.4f}'.format(grid_xgboost.best_score_))
        print( 'best parameters: {}'.format(grid_xgboost.best_params_))
    
        colsample_bytree = grid_xgboost.best_params_['colsample_bytree']
        n_estimators = grid_xgboost.best_params_['n_estimators']
        max_depth = grid_xgboost.best_params_['max_depth']
        min_child_weight = grid_xgboost.best_params_['min_child_weight']
        alpha = grid_xgboost.best_params_['alpha']
    
        xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = colsample_bytree ,
                max_depth = max_depth, alpha = alpha, n_estimators = n_estimators, min_child_weight=min_child_weight)
    
        xg_reg.fit(X_train,y_train)
        y_test_pred = xg_reg.predict(X_test)
        print('xgBoost result train data')
        print(mean_absolute_error(y_test, y_test_pred))
        print(mean_squared_error(y_test, y_test_pred))
        print(r2_score(y_test, y_test_pred))
    
        plt.figure(figsize=(16,6))
        plt.plot(y_test.values,label="Actual data")
        plt.plot(y_test_pred,label="predicted values")
        plt.legend(loc="best")
        
        fig, ax = plt.subplots(figsize=(10,10))
        import matplotlib.pyplot as plt
        xgb.plot_importance(xg_reg,importance_type='gain', height=0.5,ax=ax)
        plt.show()
        
        #we then fit the whole dataset to predict next day purchase    
        xg_reg.fit(x,y)
    
        preds_test = xg_reg.predict(test)
        
        df = pd.DataFrame()
        
        df['Sales_Predict'] = preds_test       
        df['Item'] = k
        df['Store'] = i
        
        
        appended_data.append(df)

In [ ]:
masterDF = pd.concat(appended_data, ignore_index=True)
masterDF['Sales_Predict'][masterDF['Sales_Predict'] < 0] = 0
masterDF_fill=masterDF .fillna(0)
pd.options.display.float_format = '{:,.4f}'.format

In [ ]:
appended_data_percent = []
for k in items:
    percent=masterDF_fill[masterDF_fill['Item']==k]
    percent['percent']=percent['Sales_Predict']/percent['Sales_Predict'].sum()*100
    appended_data_percent.append(percent)

In [ ]:
#sales predict
masterDF_last = pd.concat(appended_data_percent, ignore_index=True)
masterDF_last

# compare Cluster of stores with sales predict

for item 1

In [ ]:
masterDF_last_item_1=masterDF_last[masterDF_last['Item']=='Item 1']
daily_sales_mean_item_1=masterDF_last_item_1.groupby(['Store'])['Sales_Predict'].mean().reset_index()
df_item_1=df_[df_['Item']=='Item 1']
lastDF_item_1=df_item_1.merge(daily_sales_mean_item_1, on="Store", how='left')
lastDF_item_1_last=lastDF_item_1.dropna()
lastDF_item_1_last.groupby(['Cluster'])['Sales_Predict'].mean().reset_index()

For item 1 the predicted values are in consistency with cluster of stores

for item 2

In [ ]:
masterDF_last_item_2=masterDF_last[masterDF_last['Item']=='Item 2']
daily_sales_mean_item_2=masterDF_last_item_2.groupby(['Store'])['Sales_Predict'].mean().reset_index()
df_item_2=df_[df_['Item']=='Item 2']
lastDF_item_2=df_item_2.merge(daily_sales_mean_item_2, on="Store", how='left')
lastDF_item_2_last=lastDF_item_2.dropna()
lastDF_item_2_last.groupby(['Cluster'])['Sales_Predict'].mean().reset_index()

For item 2 cluster C stores prediction is higher than cluster B stores. But we should keep in mind we predict the day monday. 31 August. So we will visualize sales of item 2 based on day of week.

In [ ]:
sales2=sales[sales['Item']=='Item 2']
sales2['Date'] =  pd.to_datetime(sales2['Date'], dayfirst=True)
def create_date_table(start, end):
        df = pd.DataFrame({"Date": pd.date_range(start, end)})
        df["Amount"] = 0
        df["Day"] = df.Date.dt.day_name()
        df["DayOfWeek"] = df.Date.dt.weekday
        df["Week"] = df.Date.dt.weekofyear
        df["Quarter"] = df.Date.dt.quarter
        df["Year"] = df.Date.dt.year
        df["Month"] = df.Date.dt.month
        df["DayOfMonth"] = df.Date.dt.day
        df["Year_half"] = (df.Quarter + 1) // 2
        return df
    
datedata=create_date_table('03.01.2020','30.08.2020')    
dailysales=datedata.merge(sales2, on="Date", how='left')
dailysales2=dailysales[['Date','DayOfWeek','Week','Year','Month','DayOfMonth','Store','Quantity']]
dailysales3=dailysales2.groupby(['Date','DayOfWeek','Week','Year','Month','DayOfMonth','Store'])['Quantity'].sum().reset_index(name ='Quantity')
dailysales3.index = dailysales3['Date']    
    
start_date = pd.Timestamp('2020-05-01')
end_date = pd.Timestamp('2020-06-01')
dailysales3['Covid19'] = dailysales3['Date'].apply(lambda x: 1 if  (x > start_date) & (x <= end_date ) else 0 )
    
del dailysales3['Date']

dailysales3['Quantity']=dailysales3['Quantity'].fillna(0)
dailysales3['Weekend'] = dailysales3['DayOfWeek'].apply(lambda x: 1 if  (x == 5) | (x == 6 ) else 0 )

In [ ]:
df_item_2_cluster=df_[df_['Item']=='Item 2']
df_item_2_cluster=df_item_2_cluster[['Store','Cluster']]
dailysales3_merge=dailysales3.merge(df_item_2_cluster, on="Store", how='left')
plt.figure(figsize=(16,6))
sns.boxplot(data=dailysales3_merge,x=dailysales3_merge['DayOfWeek'],y=dailysales3_merge['Quantity'],hue=dailysales3_merge.Cluster)

Even though on monday we see the median of Cluster B is higher than Cluster C , Cluster C has outliers higher than Cluster B, so on some Mondays Cluster C value might be higher than Cluster B value. We can say that below predict is true.

In [ ]:
lastDF_item_2_last.groupby(['Cluster'])['Sales_Predict'].mean().reset_index()